Testing time of multiple part of program(faster cuda):

In [1]:
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
import time
import timeit
from pycuda.compiler import SourceModule

mod2 = SourceModule("""
__global__
void SpMV_kernel_64(double *x, double *y) 
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int R2 = blockIdx.y * blockDim.y + threadIdx.y;
    int R3 = blockIdx.z * blockDim.z + threadIdx.z;
    int NX = 128;


    double sum = 0.0;
    for (int bz = -1; bz < 2; bz++) {
        for (int by = -1; by < 2; by++) {
            for (int bx = -1; bx < 2; bx++) {
                if(R1 + bz >= 0 && R1+bz < NX && R2+by >= 0 && R2+by < NX && R3+bx >= 0 && R3+bx < NX)
                sum += x[(R1+bz)*NX*NX+(R2+by)*NX+(R3+bx)];
            }
        }
    }
    y[(R1) * (NX) * (NX) + (R2) * (NX) + R3] = -sum + 27.0 * x[(R1) * (NX) * (NX) + (R2) * (NX) + R3];
}

""")
spmv_gpu_64 = mod2.get_function("SpMV_kernel_64")

mod9 = SourceModule("""
__global__
void apxby_kernel_1d(double *x,  double *y, double *des, double *alpha, double *beta)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int ind = R1;
    des[ind] = alpha[0]*x[ind] + beta[0]*y[ind];
            
}
""")

apxby_gpu_1d = mod9.get_function("apxby_kernel_1d")


mod7 = SourceModule("""
__global__
void copy_kernel(double *src, double *dst)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    dst[R1] = src[R1];


}
""")
cpy_gpu = mod7.get_function("copy_kernel")


mod8 = SourceModule("""
__global__ void Dot_Prod(double *x, double *y, double *g_odata) {
 __shared__ double sdata[512];
// each thread loads one element from global to shared mem
unsigned int tid = threadIdx.x;
unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
sdata[tid] = x[i]*y[i];
__syncthreads();
// do reduction in shared mem
for(unsigned int s=1; s < blockDim.x; s *= 2) {
if (tid % (2*s) == 0) {
sdata[tid] += sdata[tid + s];
}
__syncthreads();
}
// write result for this block to global mem
if (tid == 0) g_odata[blockIdx.x] = sdata[0];
}


""")
mod10 = SourceModule("""
__global__ void reduce0(double *g_in, double *g_odata) {
 __shared__ double sdata[512];
// each thread loads one element from global to shared mem
unsigned int tid = threadIdx.x;
unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
sdata[tid] = g_in[i];
__syncthreads();
// do reduction in shared mem
for(unsigned int s=1; s < blockDim.x; s *= 2) {
if (tid % (2*s) == 0) {
sdata[tid] += sdata[tid + s];
}
__syncthreads();
}
// write result for this block to global mem
if (tid == 0) g_odata[blockIdx.x] = sdata[0];
}
""")
partial_dot = mod8.get_function("Dot_Prod")
partial_red = mod10.get_function("reduce0")

def cal_dpdt(a_gpu,b_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer1,cpu_buffer2):
    '''
    ans = (a,b)/(b,b)
    '''
    partial_dot(a_gpu,b_gpu,gpu_buffer1,block=(512,1,1),grid=(4096,1,1))
    partial_red(gpu_buffer1,gpu_buffer2,block=(512,1,1),grid=(8,1,1))
    
    cuda.memcpy_dtoh(cpu_buffer1,gpu_buffer2)
    cpu_buffer2[0]= np.sum(cpu_buffer1)
    
    partial_dot(b_gpu,b_gpu,gpu_buffer1,block=(512,1,1),grid=(4096,1,1))
    partial_red(gpu_buffer1,gpu_buffer2,block=(512,1,1),grid=(8,1,1))
    cuda.memcpy_dtoh(cpu_buffer1,gpu_buffer2)
    cpu_buffer2[1]= np.sum(cpu_buffer1)
    
    return cpu_buffer2[0]/cpu_buffer2[1]

mod11 = SourceModule("""
__global__
void plus_alpha_vector(double *x,  double *y, double *alpha)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int ind = R1;
    x[ind] = x[ind] + alpha[0]*y[ind];
            
}
""")

mod12 = SourceModule("""
__global__
void subtract_alpha_vector(double *x,  double *y, double *alpha)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int ind = R1;
    x[ind] = x[ind] - alpha[0]*y[ind];
            
}
""")

sub_alpha_gpu = mod12.get_function("subtract_alpha_vector")

plus_alpha_gpu = mod11.get_function("plus_alpha_vector")


def gpu_minres_v2(x0,b,maxit):
    '''
    gpu acceleration of MINRES algorithm
    x0 ,b: numpy.float64 arrays of initial guess and coefficient vector
    maxit: maximum number of iterations
    '''   
    alloctime = 0.0
    cpytime = 0.0
    kerneltime=0.0
    
    #event1 = cuda.Event()
    #event2 = cuda.Event()
    #event3 = cuda.Event()
    #event4 = cuda.Event()
    #event5 = cuda.Event()
    #event6 = cuda.Event()



    #event1.record()
    numpytime = 0.0
    gputime = 0.0
    
    time1 = time.perf_counter()
    
    t1 = time.perf_counter()

    #allocating memory
    #ans=np.zeros_like(x0).astype(np.float64)
    x=np.zeros_like(x0).astype(np.float64)
    r=np.zeros_like(x0).astype(np.float64)


    cst_one=np.array([1.0]).astype(np.float64)
    cst_neone=np.array([-1.0]).astype(np.float64)
    
    #alpha=np.array([1.0]).astype(np.float64)
    #beta1=np.array([1.0]).astype(np.float64)
    #beta2=np.array([1.0]).astype(np.float64)
    
    cpu_buffer_1=np.zeros(8).astype(np.float64)
    cpu_buffer_2f=np.zeros(2).astype(np.float64)
    cpu_buffer_3=np.zeros(1).astype(np.float64)

    numpytime = numpytime + time.perf_counter()-t1
    print(numpytime)
    
    t2 = time.perf_counter()
    
    x0_gpu = cuda.mem_alloc(x0.nbytes)
    x_gpu = cuda.mem_alloc(x0.nbytes)
    b_gpu = cuda.mem_alloc(x0.nbytes)
    r_gpu = cuda.mem_alloc(x0.nbytes)
    p0_gpu = cuda.mem_alloc(x0.nbytes)
    p1_gpu = cuda.mem_alloc(x0.nbytes)
    p2_gpu = cuda.mem_alloc(x0.nbytes)

    
    s0_gpu = cuda.mem_alloc(x0.nbytes)
    s1_gpu = cuda.mem_alloc(x0.nbytes)
    s2_gpu = cuda.mem_alloc(x0.nbytes)

    
    one_gpu = cuda.mem_alloc(cst_one.nbytes)
    neone_gpu = cuda.mem_alloc(cst_one.nbytes)
    
    alpha_gpu = cuda.mem_alloc(8)
    beta1_gpu = cuda.mem_alloc(8)
    beta2_gpu = cuda.mem_alloc(8)

    
    gpu_buffer1 = cuda.mem_alloc(int(x0.nbytes/512))
    gpu_buffer2 = cuda.mem_alloc(int(x0.nbytes/(512*512)))
    
    temp = cuda.mem_alloc(x0.nbytes)
    
    gputime = gputime+ time.perf_counter()-t2
    print(gputime)
    
    #event2.record()
    #event2.synchronize()
    #alloctime = alloctime + event2.time_since(event1)
    
    
    #print("allocating completed")
    
    #event3.record()

    alloctime = alloctime + time.perf_counter()-time1
    time2 = time.perf_counter()
    #copy data
    cuda.memcpy_htod(x0_gpu, x0)
    cuda.memcpy_htod(b_gpu, b)

    cuda.memcpy_htod(one_gpu, cst_one)
    cuda.memcpy_htod(neone_gpu, cst_neone)
    
    cpytime = cpytime + time.perf_counter()-time2
    
    #event4.record()
    #event4.synchronize()
    #cpytime = cpytime + event4.time_since(event3)

    #print("memory copy completed, begin kernel computing")
    
    #event5.record()
    #x = np.array(x0)
    time3 = time.perf_counter()
    
    cpy_gpu(x0_gpu,x_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #r = b - A @ x0
    spmv_gpu_64(x0_gpu,temp,block=(8,8,8), grid=(16,16,16))
    
    apxby_gpu_1d(
        b_gpu,temp,r_gpu,one_gpu,neone_gpu,
        block=(512,1,1), grid=(16*16*16,1,1))
    
    #p0 = np.array(r)
    cpy_gpu(r_gpu,p0_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s0 = A @ p0
    spmv_gpu_64(p0_gpu,s0_gpu,block=(8,8,8), grid=(16,16,16))
    
    #p1 = np.array(p0)
    cpy_gpu(p0_gpu,p1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s1 = np.array(s0)
    cpy_gpu(s0_gpu,s1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #print("pre-loop completed, begin iters...")
    
    #event6.record()
    #event6.synchronize()
    #kerneltime = kerneltime + event6.time_since(event5)
    
    kerneltime =  kerneltime + time.perf_counter()- time3
    
    for iter in range(1,maxit):
        
        time5 = time.perf_counter()
        #exec(f'eventa{iter}=cuda.Event()')
        #exec(f'eventb{iter} = cuda.Event()')

        #event7 = cuda.Event()
        #event8 = cuda.Event()
        
        #exec(f'eventa{iter}.record()')
    
        #p2 = np.ndarray.copy(p1)
        cpy_gpu(p1_gpu,p2_gpu,block=(512,1,1),grid=(16*16*16,1,1))

        #p1 = np.ndarray.copy(p0)
        cpy_gpu(p0_gpu,p1_gpu,block=(512,1,1),grid=(16*16*16,1,1))

        #s2 = np.ndarray.copy(s1)
        cpy_gpu(s1_gpu,s2_gpu,block=(512,1,1),grid=(16*16*16,1,1))

        #s1 = np.ndarray.copy(s0)
        cpy_gpu(s0_gpu,s1_gpu,block=(512,1,1),grid=(16*16*16,1,1))


        #alpha = np.dot(r,s1)/np.dot(s1,s1)
        cpu_buffer_3[0] = cal_dpdt(r_gpu,s1_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
        cuda.memcpy_htod(alpha_gpu, cpu_buffer_3)

        #x = x + alpha * p1
        plus_alpha_gpu(x_gpu,p1_gpu,alpha_gpu,block=(512,1,1),grid=(4096,1,1))

        #r = r - alpha * s1
        sub_alpha_gpu(r_gpu,s1_gpu,alpha_gpu,block=(512,1,1),grid=(4096,1,1))


        #p0 = np.ndarray.copy(s1)
        cpy_gpu(s1_gpu,p0_gpu,block=(512,1,1),grid=(16*16*16,1,1))


        #s0 = A @ s1
        spmv_gpu_64(s1_gpu,s0_gpu,block=(8,8,8), grid=(16,16,16))

        #beta1 = np.dot(s0,s1)/np.dot(s1,s1)
        cpu_buffer_3[0] = cal_dpdt(s0_gpu,s1_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
        cuda.memcpy_htod(beta1_gpu, cpu_buffer_3)

        #p0 = p0 - beta1* p1
        sub_alpha_gpu(p0_gpu,p1_gpu,beta1_gpu,block=(512,1,1),grid=(4096,1,1))


        #s0 = s0 - beta1* s1
        sub_alpha_gpu(s0_gpu,s1_gpu,beta1_gpu,block=(512,1,1),grid=(4096,1,1))

        if iter>1:

                #beta2 = np.dot(s0,s2)/np.dot(s2,s2)
                cpu_buffer_3[0] = cal_dpdt(s0_gpu,s2_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
                cuda.memcpy_htod(beta2_gpu, cpu_buffer_3)

                #p0 = p0 - beta2* p2
                sub_alpha_gpu(p0_gpu,p2_gpu,beta2_gpu,block=(512,1,1),grid=(4096,1,1))

                #s0 = s0 - beta2* s2
                sub_alpha_gpu(s0_gpu,s2_gpu,beta2_gpu,block=(512,1,1),grid=(4096,1,1))
                
        kerneltime = kerneltime + time.perf_counter()-time5       

        #event8.record()
        #event8.synchronize()
        #kerneltime = kerneltime + event8.time_since(event7)

        #exec(f'eventb{iter}.record()')
        #exec(f'eventb{iter}.synchronize()')
        #exec(f'kerneltime=kerneltime + eventb{iter}.time_since(eventa{iter})')



    
    
    #print("calculation completed, writing back...")
    #event9 = cuda.Event()
    #event10=cuda.Event()
    #event9.record()
    
    time6=time.perf_counter()
    
    cuda.memcpy_dtoh(x,x_gpu)
    cuda.memcpy_dtoh(r,r_gpu)
    
    cpytime == cpytime + time.perf_counter()-time6
    
    #event10.record()
    #event10.synchronize()
    #cpytime = cpytime + event10.time_since(event9)
    #print("finished")
    return x,r, alloctime,cpytime,kerneltime


x_exact = np.ones(128*128*128).astype(np.float64)
x0 = np.zeros(128*128*128).astype(np.float64)
b = np.random.random(128*128*128).astype(np.float64)

spmv_gpu_64(cuda.In(x_exact),cuda.InOut(b),block=(8,8,8),grid=(16,16,16))

st = time.perf_counter()
[x,r,t1,t2,t3]= gpu_minres_v2(x0,b,50)
print(t1,t2,t3)
sp = (time.perf_counter()-st)

relative_error = np.sqrt(np.sum(np.square(x-x_exact)))/np.sqrt(np.sum(np.square(x_exact)))
residue = np.sqrt(np.sum(np.square(r)))

print("relative error={}, residue={}".format(relative_error,residue))
print("elapsed time={}".format(sp))





ModuleNotFoundError: No module named 'pycuda'

Testing time of multiple part of program(cuda):

In [84]:
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
#import time
#import timeit
from pycuda.compiler import SourceModule

mod2 = SourceModule("""
__global__
void SpMV_kernel_64(double *x, double *y) 
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int R2 = blockIdx.y * blockDim.y + threadIdx.y;
    int R3 = blockIdx.z * blockDim.z + threadIdx.z;
    int NX = 128;


    double sum = 0.0;
    for (int bz = -1; bz < 2; bz++) {
        for (int by = -1; by < 2; by++) {
            for (int bx = -1; bx < 2; bx++) {
                if(R1 + bz >= 0 && R1+bz < NX && R2+by >= 0 && R2+by < NX && R3+bx >= 0 && R3+bx < NX)
                sum += x[(R1+bz)*NX*NX+(R2+by)*NX+(R3+bx)];
            }
        }
    }
    y[(R1) * (NX) * (NX) + (R2) * (NX) + R3] = -sum + 27.0 * x[(R1) * (NX) * (NX) + (R2) * (NX) + R3];
}

""")
spmv_gpu_64 = mod2.get_function("SpMV_kernel_64")

mod9 = SourceModule("""
__global__
void apxby_kernel_1d(double *x,  double *y, double *des, double *alpha, double *beta)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int ind = R1;
    des[ind] = alpha[0]*x[ind] + beta[0]*y[ind];
            
}
""")

apxby_gpu_1d = mod9.get_function("apxby_kernel_1d")


mod7 = SourceModule("""
__global__
void copy_kernel(double *src, double *dst)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    dst[R1] = src[R1];


}
""")
cpy_gpu = mod7.get_function("copy_kernel")


mod8 = SourceModule("""
__global__ void Dot_Prod(double *x, double *y, double *g_odata) {
 __shared__ double sdata[512];
// each thread loads one element from global to shared mem
unsigned int tid = threadIdx.x;
unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
sdata[tid] = x[i]*y[i];
__syncthreads();
// do reduction in shared mem
for(unsigned int s=1; s < blockDim.x; s *= 2) {
if (tid % (2*s) == 0) {
sdata[tid] += sdata[tid + s];
}
__syncthreads();
}
// write result for this block to global mem
if (tid == 0) g_odata[blockIdx.x] = sdata[0];
}


""")
mod10 = SourceModule("""
__global__ void reduce0(double *g_in, double *g_odata) {
 __shared__ double sdata[512];
// each thread loads one element from global to shared mem
unsigned int tid = threadIdx.x;
unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
sdata[tid] = g_in[i];
__syncthreads();
// do reduction in shared mem
for(unsigned int s=1; s < blockDim.x; s *= 2) {
if (tid % (2*s) == 0) {
sdata[tid] += sdata[tid + s];
}
__syncthreads();
}
// write result for this block to global mem
if (tid == 0) g_odata[blockIdx.x] = sdata[0];
}
""")
partial_dot = mod8.get_function("Dot_Prod")
partial_red = mod10.get_function("reduce0")

def cal_dpdt(a_gpu,b_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer1,cpu_buffer2):
    '''
    ans = (a,b)/(b,b)
    '''
    partial_dot(a_gpu,b_gpu,gpu_buffer1,block=(512,1,1),grid=(4096,1,1))
    partial_red(gpu_buffer1,gpu_buffer2,block=(512,1,1),grid=(8,1,1))
    
    cuda.memcpy_dtoh(cpu_buffer1,gpu_buffer2)
    cpu_buffer2[0]= np.sum(cpu_buffer1)
    
    partial_dot(b_gpu,b_gpu,gpu_buffer1,block=(512,1,1),grid=(4096,1,1))
    partial_red(gpu_buffer1,gpu_buffer2,block=(512,1,1),grid=(8,1,1))
    cuda.memcpy_dtoh(cpu_buffer1,gpu_buffer2)
    cpu_buffer2[1]= np.sum(cpu_buffer1)
    
    return cpu_buffer2[0]/cpu_buffer2[1]

mod11 = SourceModule("""
__global__
void plus_alpha_vector(double *x,  double *y, double *alpha)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int ind = R1;
    x[ind] = x[ind] + alpha[0]*y[ind];
            
}
""")

mod12 = SourceModule("""
__global__
void subtract_alpha_vector(double *x,  double *y, double *alpha)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int ind = R1;
    x[ind] = x[ind] - alpha[0]*y[ind];
            
}
""")

sub_alpha_gpu = mod12.get_function("subtract_alpha_vector")

plus_alpha_gpu = mod11.get_function("plus_alpha_vector")


def gpu_minres_v2(x0,b,maxit,x,r):
    '''
    gpu acceleration of MINRES algorithm
    x0 ,b: numpy.float64 arrays of initial guess and coefficient vector
    maxit: maximum number of iterations
    '''   
    
    #allocating memory
    #ans=np.zeros_like(x0).astype(np.float64)
    #x=np.zeros_like(x0).astype(np.float64)
    #r=np.zeros_like(x0).astype(np.float64)


    cst_one=np.array([1.0]).astype(np.float64)
    cst_neone=np.array([-1.0]).astype(np.float64)
    
    alpha=np.array([1.0]).astype(np.float64)
    beta1=np.array([1.0]).astype(np.float64)
    beta2=np.array([1.0]).astype(np.float64)



    
    x0_gpu = cuda.mem_alloc(x0.nbytes)
    x_gpu = cuda.mem_alloc(x0.nbytes)
    b_gpu = cuda.mem_alloc(x0.nbytes)
    r_gpu = cuda.mem_alloc(x0.nbytes)
    p0_gpu = cuda.mem_alloc(x0.nbytes)
    p1_gpu = cuda.mem_alloc(x0.nbytes)
    p2_gpu = cuda.mem_alloc(x0.nbytes)

    
    s0_gpu = cuda.mem_alloc(x0.nbytes)
    s1_gpu = cuda.mem_alloc(x0.nbytes)
    s2_gpu = cuda.mem_alloc(x0.nbytes)

    
    one_gpu = cuda.mem_alloc(cst_one.nbytes)
    neone_gpu = cuda.mem_alloc(cst_one.nbytes)
    
    alpha_gpu = cuda.mem_alloc(alpha.nbytes)
    beta1_gpu = cuda.mem_alloc(beta1.nbytes)
    beta2_gpu = cuda.mem_alloc(beta2.nbytes)

    
    gpu_buffer1 = cuda.mem_alloc(int(x0.nbytes/512))
    gpu_buffer2 = cuda.mem_alloc(int(x0.nbytes/(512*512)))
    
    cpu_buffer_1=np.zeros(8).astype(np.float64)
    cpu_buffer_2f=np.zeros(2).astype(np.float64)
    cpu_buffer_3=np.zeros(1).astype(np.float64)


    
    temp = cuda.mem_alloc(x0.nbytes)
    
    #print("allocating completed")
    
    #copy data
    cuda.memcpy_htod(x0_gpu, x0)
    cuda.memcpy_htod(b_gpu, b)

    cuda.memcpy_htod(one_gpu, cst_one)
    cuda.memcpy_htod(neone_gpu, cst_neone)
    
    #print("memory copy completed, begin kernel computing")

    #x = np.array(x0)
    cpy_gpu(x0_gpu,x_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #r = b - A @ x0
    spmv_gpu_64(x0_gpu,temp,block=(4,8,8), grid=(32,16,16))
    
    apxby_gpu_1d(
        b_gpu,temp,r_gpu,one_gpu,neone_gpu,
        block=(512,1,1), grid=(16*16*16,1,1))
    
    #p0 = np.array(r)
    cpy_gpu(r_gpu,p0_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s0 = A @ p0
    spmv_gpu_64(p0_gpu,s0_gpu,block=(4,8,8), grid=(32,16,16))
    
    #p1 = np.array(p0)
    cpy_gpu(p0_gpu,p1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s1 = np.array(s0)
    cpy_gpu(s0_gpu,s1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #print("pre-loop completed, begin iters...")
    
    for iter in range(1,maxit):
    
        #p2 = np.ndarray.copy(p1)
        cpy_gpu(p1_gpu,p2_gpu,block=(512,1,1),grid=(16*16*16,1,1))

        #p1 = np.ndarray.copy(p0)
        cpy_gpu(p0_gpu,p1_gpu,block=(512,1,1),grid=(16*16*16,1,1))

        #s2 = np.ndarray.copy(s1)
        cpy_gpu(s1_gpu,s2_gpu,block=(512,1,1),grid=(16*16*16,1,1))

        #s1 = np.ndarray.copy(s0)
        cpy_gpu(s0_gpu,s1_gpu,block=(512,1,1),grid=(16*16*16,1,1))


        #alpha = np.dot(r,s1)/np.dot(s1,s1)
        cpu_buffer_3[0] = cal_dpdt(r_gpu,s1_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
        cuda.memcpy_htod(alpha_gpu, cpu_buffer_3)

        #x = x + alpha * p1
        plus_alpha_gpu(x_gpu,p1_gpu,alpha_gpu,block=(512,1,1),grid=(4096,1,1))

        #r = r - alpha * s1
        sub_alpha_gpu(r_gpu,s1_gpu,alpha_gpu,block=(512,1,1),grid=(4096,1,1))


        #p0 = np.ndarray.copy(s1)
        cpy_gpu(s1_gpu,p0_gpu,block=(512,1,1),grid=(16*16*16,1,1))


        #s0 = A @ s1
        spmv_gpu_64(s1_gpu,s0_gpu,block=(4,8,8), grid=(32,16,16))

        #beta1 = np.dot(s0,s1)/np.dot(s1,s1)
        cpu_buffer_3[0] = cal_dpdt(s0_gpu,s1_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
        cuda.memcpy_htod(beta1_gpu, cpu_buffer_3)

        #p0 = p0 - beta1* p1
        sub_alpha_gpu(p0_gpu,p1_gpu,beta1_gpu,block=(512,1,1),grid=(4096,1,1))


        #s0 = s0 - beta1* s1
        sub_alpha_gpu(s0_gpu,s1_gpu,beta1_gpu,block=(512,1,1),grid=(4096,1,1))

        if iter>1:

                #beta2 = np.dot(s0,s2)/np.dot(s2,s2)
                cpu_buffer_3[0] = cal_dpdt(s0_gpu,s2_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
                cuda.memcpy_htod(beta2_gpu, cpu_buffer_3)

                #p0 = p0 - beta2* p2
                sub_alpha_gpu(p0_gpu,p2_gpu,beta2_gpu,block=(512,1,1),grid=(4096,1,1))

                #s0 = s0 - beta2* s2
                sub_alpha_gpu(s0_gpu,s2_gpu,beta2_gpu,block=(512,1,1),grid=(4096,1,1))

    
    
    #print("calculation completed, writing back...")
    
    cuda.memcpy_dtoh(x,x_gpu)
    cuda.memcpy_dtoh(r,r_gpu)

    #print("finished")
    return 


x_exact = np.ones(128*128*128).astype(np.float64)

x = np.ones(128*128*128).astype(np.float64)
r = np.ones(128*128*128).astype(np.float64)

x0 = np.zeros(128*128*128).astype(np.float64)
b = np.random.random(128*128*128).astype(np.float64)

spmv_gpu_64(cuda.In(x_exact),cuda.InOut(b),block=(8,8,8),grid=(16,16,16))

st = time.perf_counter()
gpu_minres_v2(x0,b,50,x,r)
sp = (time.perf_counter()-st)

relative_error = np.sqrt(np.sum(np.square(x-x_exact)))/np.sqrt(np.sum(np.square(x_exact)))
residue = np.sqrt(np.sum(np.square(r)))

print("relative error={}, residue={}".format(relative_error,residue))
print("elapsed time={}".format(sp))





relative error=0.45699109371422114, residue=12.604018126016776
elapsed time=0.18545212499884656


mem alloc time:0.0244211630006248 cpu-gpu copy time:0.007241348000206926 kernel time: 0.1790220609982498
relative error=0.45699109371422114, residue=12.604018126016776
elapsed time=0.22552574499968614

details of alloc time:
numpy time: 0.013281465000545722
gpu time: 0.001474357000006421


Testing specific kernel:

In [ ]:


from pycuda.compiler import SourceModule
mod = SourceModule("""
__global__
void apxby_kernel_64(double *x,  double *y, double *des, double *alpha, double *beta)
{
    int NX = 128;
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int R2 = blockIdx.y * blockDim.y + threadIdx.y;
    int R3 = blockIdx.z * blockDim.z + threadIdx.z;
    
    
    int ind = (R1) * (NX) * (NX) + (R2) * (NX) + R3;
    des[ind] = alpha[0]*x[ind] + beta[0]*y[ind];
            
}
""")

apxby_gpu_64 = mod.get_function("apxby_kernel_64")

a = np.random.randn(128*128*128).astype(np.float64)
b = np.random.randn(128*128*128).astype(np.float64)
alpha = np.array([1.0]).astype(np.float64)
beta = np.array([2.0]).astype(np.float64)
dest = np.zeros_like(a).astype(np.float64)

begin = time.perf_counter()

apxby_gpu_64(
        cuda.In(a),cuda.In(b),cuda.InOut(dest),cuda.In(alpha),cuda.In(beta),
        block=(8,8,8), grid=(16,16,16))
end = time.perf_counter()

gpu_time = end-begin
print(gpu_time)

In [10]:
x_exact = np.zeros(128*128*128).astype(np.float64)

t1 = time.perf_counter()
A @ x_exact
print(time.perf_counter()-t1)

t2 = time.perf_counter()
x_gpu = cuda.mem_alloc(x_exact.nbytes)
x2 = cuda.mem_alloc(x_exact.nbytes)
cuda.memcpy_htod(x_gpu,x_exact)
spmv_gpu_64(x2,x_gpu,block=(8,8,8), grid=(16,16,16))
cuda.memcpy_dtoh(x_exact,x2)

print(time.perf_counter()-t2)






0.07263612099995953
0.007991972000127134


Comparing CPU and GPU:

In [ ]:
def cpu_apxby(a,b):
    return a*alpha[0]+b*beta[0]
begin = time.time()
c_dest = cpu_apxby(a,b)
end = time.time()
cpu_time = end - begin

print(np.max(np.abs(c_dest-dest)))
print("gpu time: {}".format(gpu_time))
print("cpu time: {}".format(cpu_time))

Testing specific kernel:

In [16]:
mod2 = SourceModule("""
__global__
void SpMV_kernel_64(double *x, double *y) 
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int R2 = blockIdx.y * blockDim.y + threadIdx.y;
    int R3 = blockIdx.z * blockDim.z + threadIdx.z;
    int NX = 128;


    double sum = 0.0;
    for (int bz = -1; bz < 2; bz++) {
        for (int by = -1; by < 2; by++) {
            for (int bx = -1; bx < 2; bx++) {
                if(R1 + bz >= 0 && R1+bz < NX && R2+by >= 0 && R2+by < NX && R3+bx >= 0 && R3+bx < NX)
                sum += x[(R1+bz)*NX*NX+(R2+by)*NX+(R3+bx)];
            }
        }
    }
    y[(R1) * (NX) * (NX) + (R2) * (NX) + R3] = -sum + 27.0 * x[(R1) * (NX) * (NX) + (R2) * (NX) + R3];
}

""",arch='sm_60 -O3')
spmv_gpu_64_fast = mod2.get_function("SpMV_kernel_64")
e = np.random.randn(128*128*128).astype(np.float64)
f = np.zeros_like(e)
start = time.perf_counter()
spmv_gpu_64_fast(cuda.In(e),cuda.InOut(f),block=(8,8,8), grid=(16,16,16))

end = time.perf_counter()

gpu_time = end-start
print(gpu_time)

CompileError: nvcc compilation of /tmp/tmp8f41ev4e/kernel.cu failed
[command: nvcc --cubin -arch sm_60 -O3 -I/opt/conda/lib/python3.7/site-packages/pycuda/cuda kernel.cu]
[stderr:
nvcc fatal   : Value 'sm_60 -O3' is not defined for option 'gpu-architecture'
]

In [11]:
def testspmv(block,grid):
    a = np.random.random(128*128*128).astype(np.float64)
    b = np.zeros_like(a)
    a_gpu = cuda.mem_alloc(a.nbytes)
    b_gpu = cuda.mem_alloc(b.nbytes)
    cuda.memcpy_htod(a_gpu,a)
    #t1 = time.perf_counter()
    %timeit spmv_gpu_64_fast(a_gpu,b_gpu,block=block,grid=grid)
    #t2 = time.perf_counter()
    #print(block, t2-t1)
    return
print("rn")


rn


In [12]:
testspmv((8,8,8),(16,16,16))

825 µs ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


824 µs ± 21.4 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [57]:
testspmv((4,8,8),(32,16,16))

812 µs ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [58]:
testspmv((16,8,8),(8,16,16))

919 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [59]:
testspmv((4,4,4),(32,32,32))

901 µs ± 26.7 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Making sparse matrix\Tesing cpu:

In [5]:
from scipy.sparse import dia_matrix
total =128*128*128
N = 128
data = 26*np.ones(total)
offsets = np.array([0])
A = dia_matrix((data,offsets),shape=(total,total),dtype="float64")
A = A.tolil()
for i in range(N):
    for j in range(N):
        for k in range(N):
            for bz in range(-1,2):
                for by in range(-1,2):
                    for bx in range(-1,2):
                        if i+bz>=0 and i+bz < N and j + by>=0 and j+by<N and k+bx>=0 and k+bx<N and bx*bx+by*by+bz*bz!=0:
                            A[i*N*N+ j*N+k,(i+bz)*N*N+ (j+by)*N+(k+bx)] = -1   
A = A.tocsr()
x_exact = np.ones(total,dtype='float64')
x0 = np.zeros(total,dtype='float64')
b = A @ x_exact

In [7]:
def cpu_minres(A,x0,b,maxit):
    x = np.array(x0)
    r = b - A @ x0
    p0 = np.array(r)
    s0 = A @ p0
    p1 = np.array(p0)
    s1 = np.array(s0)
    for iter in range(1,maxit):
        p2 = np.ndarray.copy(p1)
        p1 = np.ndarray.copy(p0)
        s2 = np.ndarray.copy(s1)
        s1 = np.ndarray.copy(s0)
        alpha = np.dot(r,s1)/np.dot(s1,s1)
        x = x + alpha * p1
        r = r - alpha * s1
        p0 = np.ndarray.copy(s1)
        s0 = A @ s1
        beta1 = np.dot(s0,s1)/np.dot(s1,s1)
        p0 = p0 - beta1* p1
        s0 = s0 - beta1* s1
        if iter>1:
            beta2 = np.dot(s0,s2)/np.dot(s2,s2)
            p0 = p0 - beta2* p2
            s0 = s0 - beta2* s2
        #print("iter{} finished!,x0={},beta1={}".format(iter,x[0],beta1))
            
    return x, r

In [6]:
A

<2097152x2097152 sparse matrix of type '<class 'numpy.float64'>'
	with 55742968 stored elements in Compressed Sparse Row format>

In [8]:
x_exact = np.ones(128*128*128).astype(np.float64)
b = A @ x_exact
x0 = np.zeros_like(x_exact).astype(np.float64)
st = time.time()
[x,r]= cpu_minres(A,x0,b,50)
sp = time.time()-st
relative_error = np.sqrt(np.sum(np.square(x-x_exact)))/np.sqrt(np.sum(np.square(x_exact)))
residue = np.sqrt(np.sum(np.square(r)))
print("relative error={}, residue={}".format(relative_error,residue))
print("elapsed time:{} ".format(sp))

relative error=0.4569910937141935, residue=12.604018126018175
elapsed time:10.46273946762085 


Testing gpu array:

In [ ]:
import pycuda.gpuarray as gpy

t = np.random.random(128*128*128).astype(np.float64)
s = np.zeros_like(t)
k = np.zeros_like(t)
t_gpu = gpy.to_gpu(t)
s_gpu = gpy.to_gpu(s)
l = t_gpu.get()
z = s_gpu.get()
spmv_gpu_64(l,z,block=(8,8,8), grid=(16,16,16))
print("finished")


In [ ]:
print("start")
mod9 = SourceModule("""
__global__
void apxby_kernel_1d(double *x,  double *y, double *des, double *alpha, double *beta)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int ind = R1;
    des[ind] = alpha[0]*x[ind] + beta[0]*y[ind];
            
}
""")

apxby_gpu_1d = mod9.get_function("apxby_kernel_1d")

a = np.random.randn(128*128*128).astype(np.float64)
b = np.random.randn(128*128*128).astype(np.float64)
alpha = np.array([1.0]).astype(np.float64)
beta = np.array([2.0]).astype(np.float64)
dest = np.zeros_like(a).astype(np.float64)
ans =np.zeros_like(dest)
t1 = time.time()
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
alpha_gpu = cuda.mem_alloc(alpha.nbytes)
beta_gpu = cuda.mem_alloc(beta.nbytes)
dest_gpu = cuda.mem_alloc(dest.nbytes)

cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(alpha_gpu, alpha)
cuda.memcpy_htod(beta_gpu, beta)

cuda.memcpy_htod(dest_gpu, dest)

t2=time.time()
apxby_gpu_1d(
        a_gpu,b_gpu,dest_gpu,alpha_gpu,beta_gpu,
        block=(512,1,1), grid=(16*16*16,1,1))
t3=time.time()
cuda.memcpy_dtoh(ans,dest_gpu)

t4 = time.time()

print(t3-t2)
print(t4-t1)
print(np.max(np.abs(alpha[0]*a+beta[0]*b-ans)))

In [ ]:
mod7 = SourceModule("""
__global__
void copy_kernel(double *src, double *dst)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    dst[R1] = src[R1];


}
""")
cpy_gpu = mod7.get_function("copy_kernel")
a = np.random.random(128*128*128).astype(np.float64)
b = np.zeros_like(a)
cpy_gpu(cuda.In(a),cuda.InOut(b),block=(512,1,1),grid=(16*16*16,1,1))
print(a[0],b[0])


In [6]:
mod7 = SourceModule("""
__global__
void copy_kernel(double *src, double *dst)
{
    
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    dst[R1] = src[R1];


}
"""
,arch='sm_60')
cpy_gpu_1d = mod7.get_function("copy_kernel")

In [7]:
def testcpykernel(block,grid):
    a = np.random.random(128*128*128).astype(np.float64)
    b = np.zeros_like(a)
    a_gpu = cuda.mem_alloc(a.nbytes)
    b_gpu = cuda.mem_alloc(b.nbytes)
    cuda.memcpy_htod(a_gpu,a)
    t1 = time.perf_counter()
    %timeit cpy_gpu_1d(a_gpu,b_gpu,block=(block,1,1),grid=(grid,1,1))
    t2 = time.perf_counter()
    #print(block, t2-t1)
    return
print("rn")

rn


In [51]:
try:
    testcpykernel((8,8,8),(16,16,16))
except:
    raise Exception('fail')


338 µs ± 24 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [53]:
try:
    testcpykernel((4,4,4),(32,32,32))
except:
    raise Exception('fail')

379 µs ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [67]:
testcpykernel(512,4096)


64.1 µs ± 6.55 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [12]:
try:
    testcpykernel(900)
except:
    raise Exception("can't")

64.2 µs ± 7.23 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
try:
    testcpykernel(1024)
except:
    raise Exception("can't")

64.3 µs ± 4.19 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
try:
    testcpykernel(256)
except:
    raise Exception("can't")

64.1 µs ± 6.23 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
try:
    testcpykernel(128)
except:
    raise Exception("can't")

64.1 µs ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
try:
    testcpykernel(64)
except:
    raise Exception("can't")

64.2 µs ± 7.07 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [17]:
try:
    testcpykernel(32)
except:
    raise Exception("can't")

113 µs ± 3.48 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [18]:
try:
    testcpykernel(16)
except:
    raise Exception("can't")

223 µs ± 2.18 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [24]:
try:
    testcpykernel(8)
except:
    raise Exception("can't")

443 µs ± 1.34 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [26]:
try:
    testcpykernel(4)
except:
    raise Exception("can't")

884 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [27]:
try:
    testcpykernel(2)
except:
    raise Exception("can't")

1.77 ms ± 1.26 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
try:
    testcpykernel(1)
except:
    raise Exception("can't")

3.53 ms ± 21 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
try:
    testcpykernel(1025)
except:
    raise Exception("can't")

Exception: can't

Tru precompile option:

In [ ]:
cpy_gpu.prepare('P')
a_gpu=cuda.mem_alloc(a.nbytes)
b_gpu=cuda.mem_alloc(a.nbytes)
grid=16*16*16
block=512
cpy_gpu.prepared_call(grid,block,(a_gpu,b_gpu))

In [ ]:
?cpy_gpu.prepared_call

Partial results:

In [ ]:
def gpu_minres_short(x0,b):
    #x = np.array(x0)
    #r = b - A @ x0
    #p0 = np.array(r)
    #s0 = A @ p0
    #p1 = np.array(p0)
    #s1 = np.array(s0)
    
    ans=np.zeros_like(x0).astype(np.float64)
    cst_one=np.array([1.0]).astype(np.float64)
    cst_neone=np.array([-1.0]).astype(np.float64)
    
    x0_gpu = cuda.mem_alloc(x0.nbytes)
    x_gpu = cuda.mem_alloc(x0.nbytes)
    b_gpu = cuda.mem_alloc(x0.nbytes)
    r_gpu = cuda.mem_alloc(x0.nbytes)
    p0_gpu = cuda.mem_alloc(x0.nbytes)
    s0_gpu = cuda.mem_alloc(x0.nbytes)
    p1_gpu = cuda.mem_alloc(x0.nbytes)
    s1_gpu = cuda.mem_alloc(x0.nbytes)
    
    one_gpu = cuda.mem_alloc(cst_one.nbytes)
    neone_gpu = cuda.mem_alloc(cst_one.nbytes)


    
    temp = cuda.mem_alloc(x0.nbytes)
    
    print("allocating completed")
    
    cuda.memcpy_htod(x0_gpu, x0)
    cuda.memcpy_htod(b_gpu, b)

    cuda.memcpy_htod(one_gpu, cst_one)
    cuda.memcpy_htod(neone_gpu, cst_neone)
    
    print("memory copy completed, begin kernel computing")

    
    cpy_gpu(x0_gpu,x_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    spmv_gpu_64(x0_gpu,temp,block=(8,8,8), grid=(16,16,16))
    apxby_gpu_1d(
        b_gpu,temp,r_gpu,one_gpu,neone_gpu,
        block=(512,1,1), grid=(16*16*16,1,1))
    cpy_gpu(r_gpu,p0_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    spmv_gpu_64(p0_gpu,s0_gpu,block=(8,8,8), grid=(16,16,16))
    cpy_gpu(p0_gpu,p1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    cpy_gpu(s0_gpu,s1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    print("kernel computing completed, begin writing back")

    cuda.memcpy_dtoh(ans,s1_gpu)
    
    print("finished")
    return ans






    
   

In [ ]:
p = gpu_minres_short(x0,b)
print(p[0],p[1],p[2])

In [ ]:
x_exact = np.ones(128*128*128).astype(np.float64)
b = A @ x_exact
x0 = np.zeros_like(x_exact).astype(np.float64)

In [ ]:
def cpu_minres_short(A,x0,b):
    x = np.array(x0)
    r = b - A @ x0
    p0 = np.array(r)
    s0 = A @ p0
    p1 = np.array(p0)
    s1 = np.array(s0)
    return s1

In [ ]:
t = cpu_minres_short(A,x0,b)
print(t[0],t[1],t[2])

In [ ]:
mod5 = SourceModule("""
__global__ void DotProd(double* x, double* y, double* scalar){
    extern __shared__ double cache[512];

    int tid = threadIdx.x+ blockIdx.x*blockDim.x;
    int cacheIndex = threadIdx.x;

    double temp=0;
    while (tid<128*128*128){
        temp+=x[tid]*y[tid];
        tid +=blockDim.x*gridDim.x; 
    }
    cache[cacheIndex]=temp;
    __syncthreads();

    int i=blockDim.x/2;
    while(i!=0){
        if (cacheIndex<i)
            cache[cacheIndex]+=cache[cacheIndex+i];
        __syncthreads();
        i/=2;
    }
    if(cacheIndex==0)
        scalar[blockIdx.x]=cache[cacheIndex];
}
""")
dot_prod_gpu = mod5.get_function("DotProd")
a = np.random.randn(128*128*128).astype(np.float64)
b = np.random.randn(128*128*128).astype(np.float64)
c = np.array([0.]).astype(np.float64)
dot_prod_gpu(cuda.In(a),cuda.In(b),cuda.InOut(c),block=(512,1,1),grid=(4096,1,1))
print(np.dot(a,b),c)


In [85]:
mod8 = SourceModule("""
__global__ void Dot_Prod(double *x, double *y, double *g_odata) {
 __shared__ double sdata[1024];
// each thread loads one element from global to shared mem
unsigned int tid = threadIdx.x;
unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
sdata[tid] = x[i]*y[i];
__syncthreads();
// do reduction in shared mem
for(unsigned int s=1; s < blockDim.x; s *= 2) {
if (tid % (2*s) == 0) {
sdata[tid] += sdata[tid + s];
}
__syncthreads();
}
// write result for this block to global mem
if (tid == 0) g_odata[blockIdx.x] = sdata[0];
}


""")
mod10 = SourceModule("""
__global__ void reduce0(double *g_in, double *g_odata) {
 __shared__ double sdata[1024];
// each thread loads one element from global to shared mem
unsigned int tid = threadIdx.x;
unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
sdata[tid] = g_in[i];
__syncthreads();
// do reduction in shared mem
for(unsigned int s=1; s < blockDim.x; s *= 2) {
if (tid % (2*s) == 0) {
sdata[tid] += sdata[tid + s];
}
__syncthreads();
}
// write result for this block to global mem
if (tid == 0) g_odata[blockIdx.x] = sdata[0];
}
""")
partial_dot = mod8.get_function("Dot_Prod")
partial_red = mod10.get_function("reduce0")

a = np.random.random(128*128*128).astype(np.float64)
b = np.random.random(128*128*128).astype(np.float64)
c = np.zeros(32*128).astype(np.float64)
d = np.zeros(8).astype(np.float64)
%timeit partial_dot(cuda.In(a),cuda.In(b),cuda.InOut(c),block=(512,1,1),grid=(4096,1,1))
partial_red(cuda.In(c),cuda.InOut(d),block=(512,1,1),grid=(8,1,1))
answ = np.sum(d)
real_answ = np.dot(a,b)
#print(answ,real_answ)
print(a,b,real_answ,answ)


KeyboardInterrupt: 

In [ ]:
def gpu_reduce(a_gpu,b_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer1,cpu_buffer2,gpu_buff):
    partial_dot(a_gpu,b_gpu,gpu_buffer1,block=(512,1,1),grid=(4096,1,1))
    partial_red(gpu_buffer1,gpu_buffer2,block=(512,1,1),grid=(8,1,1))
    cuda.memcpy_dtoh(cpu_buffer1,gpu_buffer2)
    cpu_buffer2[0]= np.sum(cpu_buffer1)
    cuda.memcpy_htod(gpu_buff,cpu_buffer2)
    return



    

In [71]:
def cal_dpdt_fast(a_gpu,b_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer1,cpu_buffer2):
    '''
    ans = (a,b)/(b,b)
    '''
    partial_dot(a_gpu,b_gpu,gpu_buffer1,block=(1024,1,1),grid=(2048,1,1))
    partial_red(gpu_buffer1,gpu_buffer2,block=(1024,1,1),grid=(2,1,1))
    
    cuda.memcpy_dtoh(cpu_buffer1,gpu_buffer2)
    cpu_buffer2[0]= np.sum(cpu_buffer1)
    
    partial_dot(b_gpu,b_gpu,gpu_buffer1,block=(512,1,1),grid=(4096,1,1))
    partial_red(gpu_buffer1,gpu_buffer2,block=(512,1,1),grid=(8,1,1))
    cuda.memcpy_dtoh(cpu_buffer1,gpu_buffer2)
    cpu_buffer2[1]= np.sum(cpu_buffer1)
    
    return cpu_buffer2[0]/cpu_buffer2[1]


In [72]:
a = np.random.random(128*128*128).astype(np.float64)
b = np.random.random(128*128*128).astype(np.float64)
a_gpu  = cuda.mem_alloc(a.nbytes)
b_gpu  = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(a_gpu,a)
cuda.memcpy_htod(b_gpu,b)
c_gpu = cuda.mem_alloc(int(a.nbytes/512))
d_gpu = cuda.mem_alloc(int(a.nbytes/(512*512)))
d = cuda.mem_alloc(8)

c_buffer = np.zeros(8).astype(np.float64)
c2 = np.zeros(2).astype(np.float64)

%timeit ans1= cal_dpdt(a_gpu,b_gpu,c_gpu,d_gpu,c_buffer,c2)


527 µs ± 311 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
mod11 = SourceModule("""
__global__
void plus_alpha_vector(double *x,  double *y, double *alpha)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int ind = R1;
    x[ind] = x[ind] + alpha[0]*y[ind];
            
}
""")

plus_alpha_gpu = mod11.get_function("plus_alpha_vector")



In [68]:
def testalpha(block):
    a = np.random.random(128*128*128).astype(np.float64)
    b = np.random.random(128*128*128).astype(np.float64)
    alpha = np.array([3.7]).astype(np.float64)
    alpha_gpu=cuda.mem_alloc(alpha.nbytes)
    a_gpu = cuda.mem_alloc(a.nbytes)
    b_gpu = cuda.mem_alloc(a.nbytes)
    cuda.memcpy_htod(a_gpu,a)
    cuda.memcpy_htod(b_gpu,b)
    cuda.memcpy_htod(alpha_gpu,alpha)
    %timeit plus_alpha_gpu(a_gpu,b_gpu,alpha_gpu,block=(block,1,1),grid=(int(128*128*128/block),1,1))
    return


In [69]:
testalpha(512)

93.1 µs ± 8.05 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
mod12 = SourceModule("""
__global__
void subtract_alpha_vector(double *x,  double *y, double *alpha)
{
    int R1 = blockIdx.x * blockDim.x + threadIdx.x;
    int ind = R1;
    x[ind] = x[ind] - alpha[0]*y[ind];
            
}
""")

sub_alpha_gpu = mod12.get_function("subtract_alpha_vector")
a = np.random.random(128*128*128).astype(np.float64)
b = np.random.random(128*128*128).astype(np.float64)
alpha = np.array([3.7]).astype(np.float64)
c = a - alpha[0] *b
sub_alpha_gpu(cuda.InOut(a),cuda.In(b),cuda.In(alpha),block=(512,1,1),grid=(4096,1,1))
print(c[0:4])
print(a[0:4])

In [ ]:
def cpu_minres_1iter(A,x0,b):
    x = np.array(x0)
    r = b - A @ x0
    p0 = np.array(r)
    s0 = A @ p0
    p1 = np.array(p0)
    s1 = np.array(s0)
    
    p2 = np.ndarray.copy(p1)
    p1 = np.ndarray.copy(p0)
    s2 = np.ndarray.copy(s1)
    s1 = np.ndarray.copy(s0)
    alpha = np.dot(r,s1)/np.dot(s1,s1)
    x = x + alpha * p1
    r = r - alpha * s1
    p0 = np.ndarray.copy(s1)
    s0 = A @ s1
    beta1 = np.dot(s0,s1)/np.dot(s1,s1)
    p0 = p0 - beta1* p1
    s0 = s0 - beta1* s1
    
    return x, r

In [ ]:
x_exact = np.ones(128*128*128).astype(np.float64)
b = A @ x_exact
x0 = np.zeros_like(x_exact)
[x,r]=cpu_minres_1iter(A,x0,b)
print(x)
print(r)

In [ ]:
def gpu_minres_1iter(x0,b):
    #x = np.array(x0)
    #r = b - A @ x0
    #p0 = np.array(r)
    #s0 = A @ p0
    #p1 = np.array(p0)
    #s1 = np.array(s0)
    
    #allocating memory
    x=np.zeros_like(x0).astype(np.float64)
    r=np.zeros_like(x0).astype(np.float64)

    cst_one=np.array([1.0]).astype(np.float64)
    cst_neone=np.array([-1.0]).astype(np.float64)
    
    alpha=np.array([1.0]).astype(np.float64)
    beta1=np.array([1.0]).astype(np.float64)
    beta2=np.array([1.0]).astype(np.float64)



    
    x0_gpu = cuda.mem_alloc(x0.nbytes)
    x_gpu = cuda.mem_alloc(x0.nbytes)
    b_gpu = cuda.mem_alloc(x0.nbytes)
    r_gpu = cuda.mem_alloc(x0.nbytes)
    p0_gpu = cuda.mem_alloc(x0.nbytes)
    p1_gpu = cuda.mem_alloc(x0.nbytes)
    p2_gpu = cuda.mem_alloc(x0.nbytes)

    
    s0_gpu = cuda.mem_alloc(x0.nbytes)
    s1_gpu = cuda.mem_alloc(x0.nbytes)
    s2_gpu = cuda.mem_alloc(x0.nbytes)

    
    one_gpu = cuda.mem_alloc(cst_one.nbytes)
    neone_gpu = cuda.mem_alloc(cst_one.nbytes)
    
    alpha_gpu = cuda.mem_alloc(alpha.nbytes)
    beta1_gpu = cuda.mem_alloc(beta1.nbytes)
    beta2_gpu = cuda.mem_alloc(beta2.nbytes)

    
    gpu_buffer1 = cuda.mem_alloc(int(a.nbytes/512))
    gpu_buffer2 = cuda.mem_alloc(int(a.nbytes/(512*512)))
    
    cpu_buffer_1=np.zeros(8).astype(np.float64)
    cpu_buffer_2f=np.zeros(2).astype(np.float64)
    cpu_buffer_3=np.zeros(1).astype(np.float64)


    
    temp = cuda.mem_alloc(x0.nbytes)
    
    #print("allocating completed")
    
    #copy data
    cuda.memcpy_htod(x0_gpu, x0)
    cuda.memcpy_htod(b_gpu, b)

    cuda.memcpy_htod(one_gpu, cst_one)
    cuda.memcpy_htod(neone_gpu, cst_neone)
    
    #print("memory copy completed, begin kernel computing")

    #x = np.array(x0)
    cpy_gpu(x0_gpu,x_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #r = b - A @ x0
    spmv_gpu_64(x0_gpu,temp,block=(8,8,8), grid=(16,16,16))
    
    apxby_gpu_1d(
        b_gpu,temp,r_gpu,one_gpu,neone_gpu,
        block=(512,1,1), grid=(16*16*16,1,1))
    
    #p0 = np.array(r)
    cpy_gpu(r_gpu,p0_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s0 = A @ p0
    spmv_gpu_64(p0_gpu,s0_gpu,block=(8,8,8), grid=(16,16,16))
    
    #p1 = np.array(p0)
    cpy_gpu(p0_gpu,p1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s1 = np.array(s0)
    cpy_gpu(s0_gpu,s1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    print("pre-loop completed, begin iters...")
    
    #p2 = np.ndarray.copy(p1)
    cpy_gpu(p1_gpu,p2_gpu,block=(512,1,1),grid=(16*16*16,1,1))

    #p1 = np.ndarray.copy(p0)
    cpy_gpu(p0_gpu,p1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s2 = np.ndarray.copy(s1)
    cpy_gpu(s1_gpu,s2_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s1 = np.ndarray.copy(s0)
    cpy_gpu(s0_gpu,s1_gpu,block=(512,1,1),grid=(16*16*16,1,1))

    
    #alpha = np.dot(r,s1)/np.dot(s1,s1)
    cpu_buffer_3[0] = cal_dpdt(r_gpu,s1_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
    cuda.memcpy_htod(alpha_gpu, cpu_buffer_3)
    
    #x = x + alpha * p1
    plus_alpha_gpu(x_gpu,p1_gpu,alpha_gpu,block=(512,1,1),grid=(4096,1,1))
    
    #r = r - alpha * s1
    sub_alpha_gpu(r_gpu,s1_gpu,alpha_gpu,block=(512,1,1),grid=(4096,1,1))

    
    #p0 = np.ndarray.copy(s1)
    cpy_gpu(s1_gpu,p0_gpu,block=(512,1,1),grid=(16*16*16,1,1))

    
    #s0 = A @ s1
    spmv_gpu_64(s1_gpu,s0_gpu,block=(8,8,8), grid=(16,16,16))

    #beta1 = np.dot(s0,s1)/np.dot(s1,s1)
    cpu_buffer_3[0] = cal_dpdt(s0_gpu,s1_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
    cuda.memcpy_htod(beta1_gpu, cpu_buffer_3)
    
    #p0 = p0 - beta1* p1
    sub_alpha_gpu(p0_gpu,p1_gpu,beta1_gpu,block=(512,1,1),grid=(4096,1,1))

    
    #s0 = s0 - beta1* s1
    sub_alpha_gpu(s0_gpu,s1_gpu,beta1_gpu,block=(512,1,1),grid=(4096,1,1))

    
    print("calculation completed, writing back...")
    
    cuda.memcpy_dtoh(x,x_gpu)
    cuda.memcpy_dtoh(r,r_gpu)

    print("finished")
    return x,r


In [ ]:
gpu_minres_1iter(x0,b)

In [ ]:
def gpu_minres_v2(x0,b,maxit):
    #x = np.array(x0)
    #r = b - A @ x0
    #p0 = np.array(r)
    #s0 = A @ p0
    #p1 = np.array(p0)
    #s1 = np.array(s0)
    
    #allocating memory
    ans=np.zeros_like(x0).astype(np.float64)
    cst_one=np.array([1.0]).astype(np.float64)
    cst_neone=np.array([-1.0]).astype(np.float64)
    
    alpha=np.array([1.0]).astype(np.float64)
    beta1=np.array([1.0]).astype(np.float64)
    beta2=np.array([1.0]).astype(np.float64)



    
    x0_gpu = cuda.mem_alloc(x0.nbytes)
    x_gpu = cuda.mem_alloc(x0.nbytes)
    b_gpu = cuda.mem_alloc(x0.nbytes)
    r_gpu = cuda.mem_alloc(x0.nbytes)
    p0_gpu = cuda.mem_alloc(x0.nbytes)
    p1_gpu = cuda.mem_alloc(x0.nbytes)
    p2_gpu = cuda.mem_alloc(x0.nbytes)

    
    s0_gpu = cuda.mem_alloc(x0.nbytes)
    s1_gpu = cuda.mem_alloc(x0.nbytes)
    s2_gpu = cuda.mem_alloc(x0.nbytes)

    
    one_gpu = cuda.mem_alloc(cst_one.nbytes)
    neone_gpu = cuda.mem_alloc(cst_one.nbytes)
    
    alpha_gpu = cuda.mem_alloc(alpha.nbytes)
    beta1_gpu = cuda.mem_alloc(beta1.nbytes)
    beta2_gpu = cuda.mem_alloc(beta2.nbytes)

    
    gpu_buffer1 = cuda.mem_alloc(int(a.nbytes/512))
    gpu_buffer2 = cuda.mem_alloc(int(a.nbytes/(512*512)))
    
    cpu_buffer_1=np.zeros(8).astype(np.float64)
    cpu_buffer_2f=np.zeros(2).astype(np.float64)
    cpu_buffer_3=np.zeros(1).astype(np.float64)


    
    temp = cuda.mem_alloc(x0.nbytes)
    
    #print("allocating completed")
    
    #copy data
    cuda.memcpy_htod(x0_gpu, x0)
    cuda.memcpy_htod(b_gpu, b)

    cuda.memcpy_htod(one_gpu, cst_one)
    cuda.memcpy_htod(neone_gpu, cst_neone)
    
    #print("memory copy completed, begin kernel computing")

    #x = np.array(x0)
    cpy_gpu(x0_gpu,x_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #r = b - A @ x0
    spmv_gpu_64(x0_gpu,temp,block=(8,8,8), grid=(16,16,16))
    
    apxby_gpu_1d(
        b_gpu,temp,r_gpu,one_gpu,neone_gpu,
        block=(512,1,1), grid=(16*16*16,1,1))
    
    #p0 = np.array(r)
    cpy_gpu(r_gpu,p0_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s0 = A @ p0
    spmv_gpu_64(p0_gpu,s0_gpu,block=(8,8,8), grid=(16,16,16))
    
    #p1 = np.array(p0)
    cpy_gpu(p0_gpu,p1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    #s1 = np.array(s0)
    cpy_gpu(s0_gpu,s1_gpu,block=(512,1,1),grid=(16*16*16,1,1))
    
    print("pre-loop completed, begin iters...")
    
    for iter in range(1,maxit):
    
        #p2 = np.ndarray.copy(p1)
        cpy_gpu(p1_gpu,p2_gpu,block=(512,1,1),grid=(16*16*16,1,1))

        #p1 = np.ndarray.copy(p0)
        cpy_gpu(p0_gpu,p1_gpu,block=(512,1,1),grid=(16*16*16,1,1))

        #s2 = np.ndarray.copy(s1)
        cpy_gpu(s1_gpu,s2_gpu,block=(512,1,1),grid=(16*16*16,1,1))

        #s1 = np.ndarray.copy(s0)
        cpy_gpu(s0_gpu,s1_gpu,block=(512,1,1),grid=(16*16*16,1,1))


        #alpha = np.dot(r,s1)/np.dot(s1,s1)
        cpu_buffer_3[0] = cal_dpdt(r_gpu,s1_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
        cuda.memcpy_htod(alpha_gpu, cpu_buffer_3)

        #x = x + alpha * p1
        plus_alpha_gpu(x_gpu,p1_gpu,alpha_gpu,block=(512,1,1),grid=(4096,1,1))

        #r = r - alpha * s1
        sub_alpha_gpu(r_gpu,s1_gpu,alpha_gpu,block=(512,1,1),grid=(4096,1,1))


        #p0 = np.ndarray.copy(s1)
        cpy_gpu(s1_gpu,p0_gpu,block=(512,1,1),grid=(16*16*16,1,1))


        #s0 = A @ s1
        spmv_gpu_64(s1_gpu,s0_gpu,block=(8,8,8), grid=(16,16,16))

        #beta1 = np.dot(s0,s1)/np.dot(s1,s1)
        cpu_buffer_3[0] = cal_dpdt(s0_gpu,s1_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
        cuda.memcpy_htod(beta1_gpu, cpu_buffer_3)

        #p0 = p0 - beta1* p1
        sub_alpha_gpu(p0_gpu,p1_gpu,beta1_gpu,block=(512,1,1),grid=(4096,1,1))


        #s0 = s0 - beta1* s1
        sub_alpha_gpu(s0_gpu,s1_gpu,beta1_gpu,block=(512,1,1),grid=(4096,1,1))

        if iter>1:

                #beta2 = np.dot(s0,s2)/np.dot(s2,s2)
                cpu_buffer_3[0] = cal_dpdt(s0_gpu,s2_gpu,gpu_buffer1,gpu_buffer2,cpu_buffer_1,cpu_buffer_2f)
                cuda.memcpy_htod(beta2_gpu, cpu_buffer_3)

                #p0 = p0 - beta2* p2
                sub_alpha_gpu(p0_gpu,p2_gpu,beta2_gpu,block=(512,1,1),grid=(4096,1,1))

                #s0 = s0 - beta2* s2
                sub_alpha_gpu(s0_gpu,s2_gpu,beta2_gpu,block=(512,1,1),grid=(4096,1,1))

    
    
    #print("calculation completed, writing back...")
    
    cuda.memcpy_dtoh(x,x_gpu)
    cuda.memcpy_dtoh(r,r_gpu)

    #print("finished")
    return x,r


See GPU information:

In [ ]:
!nvidia-smi